In [1]:
""" ADMM color test. """
import numpy as np
import cv2
from scipy.fftpack import fft2, ifft2
import time
from utils.utils import D, DT

In [37]:
# Auxiliary functions
def z_solver(x, u, lambd, rho, mask, tv_type):
    w = mask * (D(x) + (1 / rho) * u)
    if tv_type == 'anisotropic':
        z = soft_threshold(w, lambd / rho)
    else:
        w_v, w_h = w[:, :, 0], w[:, :, 1]
        t = np.sqrt(w_v**2 + w_h**2)
        z = (np.atleast_3d(soft_threshold(t, lambd / rho) / (t + np.finfo(float).eps)))*w
    return z

def soft_threshold(x, kappa):
    return np.maximum(x - kappa, 0) - np.maximum(-x - kappa, 0)

def x_solver(z, u, u0, rho, deno):
    x = ifft2(fft2(u0 + rho * DT(z) - DT(u)) / deno)
    return x

"""implements ADMM for TV-L^2."""
def ADMM_3D(u0,lambd, N, \
              tv_type = "anisotropic",
                rho = 1, mu = 10, tau = 2, ground_truth = None, eps = 1e-3, channel_axis = None):
    # note: when ground truth is nonzero, eps is used to
    # ensure the quality of output image
    m, n, c = u0.shape
    assert c == 3, "color channel mismatch error."
    # Initialization
    u0_R = u0[:,:,0]
    u0_G = u0[:,:,1]
    u0_B = u0[:,:,2]
    x = np.zeros_like(u0)
    x_next = np.zeros_like(u0)
    z = np.zeros((m,n,3,2))
    z_next = np.zeros((m,n,3,2))
    u = np.zeros_like(z)

    def mask_u0(u_channel):
        z = D(u_channel)
        u = np.zeros_like(z)
        mask = np.ones_like(z)
        mask[-1, :, 0] = 0
        mask[:, -1, 1] = 0
        dh = np.array([[0, 0, 0], [-1, 1, 0], [0, 0, 0]])
        dh_pad = np.zeros((m, n))
        dh_pad[m//2:m//2+3, n//2:n//2+3] = dh 
        # horizontal difference operator
        dv = np.array([[0, -1, 0], [0, 1, 0], [0, 0, 0]])
        dv_pad = np.zeros((m, n))
        dv_pad[m//2:m//2+3, n//2:n//2+3] = dv
        # vertical difference operator
        fdh = fft2(dh_pad)
        fdv = fft2(dv_pad)
        deno = 1 + rho * np.abs(fdh)**2 + rho * np.abs(fdv)**2
        return deno, mask, u
    deno_R, mask_R, u[:,:,0,:] = mask_u0(u0_R)
    deno_G, mask_G, u[:,:,1,:] = mask_u0(u0_G)
    deno_B, mask_B, u[:,:,2,:] = mask_u0(u0_B)
    z = D(u0)
    print(z.shape)
    # print("z=",z)
    print("dtz=",DT(z))
    # tic = time.perf_counter()
    # denoise_process = []
    # Main loop
    # np.save("0_temp.npy",u0)
    """To display the denoising process, the intermediate values
    are saved to a temp.npy file. """
    # TODO: find RAM-economical solution to dynamical denoising display
    if ground_truth == None:
        # iterate for a set number of times
        for i in range(N):
            # TODO: set stopping criteria with original image
            x_next[:,:,0] = x_solver(z[:,:,0,:], u[:,:,0,:], u0_R, rho, deno_R)
            z_next[:,:,0,:] = z_solver(x_next[:,:,0], u[:,:,0,:], lambd, rho, mask_R, tv_type)
            x_next[:,:,1] = x_solver(z[:,:,1,:], u[:,:,1,:], u0_G, rho, deno_G)
            z_next[:,:,1,:] = z_solver(x_next[:,:,1], u[:,:,1,:], lambd, rho, mask_G, tv_type)
            x_next[:,:,2] = x_solver(z[:,:,2,:], u[:,:,2,:], u0_B, rho, deno_B)
            z_next[:,:,2,:] = z_solver(x_next[:,:,2], u[:,:,2,:], lambd, rho, mask_B, tv_type)

            u_next = u + rho * (D(x_next) - z_next)
            # print(DT(z))
            # z_next = np.stack([z_next_R, z_next_G, z_next_B], axis = 2)
            # print(DT(z_next))
            s = -rho * (DT(z_next - z))
            r = D(x) - z
            s_norm = np.linalg.norm(s)
            r_norm = np.linalg.norm(r)
            if r_norm > mu * s_norm:
                rho = rho * tau
            elif s_norm > mu * r_norm:
                rho = rho / tau
            if np.linalg.norm(x-x_next)/np.linalg.norm(x) < eps:
                break

            x = x_next
            z = z_next
            u = u_next

            x_temp = x.astype(np.uint8)
            os_dir = [str(i),"temp.npy"]
            os_dir = "_".join(os_dir)
            np.save(os_dir, x_temp)
            # denoise_process.append(u)
    else:
        # compare the generated image with ground truth
        # end iteration if difference smaller than eps
        # or when k exceeds maxiter
        k = 0
        while np.linalg.norm(ground_truth, x)/np.linalg.norm(x)>eps:
            x_next = x_solver(z, u, u0, rho, deno)
            z_next = z_solver(x_next, u, lambd, rho, mask, tv_type)
            u_next = u + rho * (D(x_next) - z_next)
            
            s = -rho * (DT(z_next - z))
            r = D(x) - z
            
            s_norm = np.linalg.norm(s)
            r_norm = np.linalg.norm(r)
            
            if r_norm > mu * s_norm:
                rho = rho * tau
            elif s_norm > mu * r_norm:
                rho = rho / tau
            
            x = x_next
            z = z_next
            u = u_next
            x_temp = x.astype(np.uint8)
            os_dir = [str(k),"temp.npy"]
            os_dir = "_".join(os_dir)
            np.save(os_dir, x_temp)
            # denoise_process.append(u)
            if k > N:
                break
            k+=1
    # toc = time.perf_counter()
    # runtime = toc - tic
    
    return x

In [38]:
if __name__ == "__main__":
    fileName = "input.png"
    N = 100
    weight = 20

    u = cv2.imread(r"E:\\2nd-semester\\practical\\test_v1\\reformulate_v1\\input_color.jpg")
    #print(u)
    u = u.astype(np.float32)#/ 255.0

    result = ADMM_3D(u, weight, N)
    #result *= 255
    result = result.astype(np.uint8)
    cv2.imwrite("E:\\2nd-semester\\practical\\test_v1\\reformulate_v1\\output_ADMM_color.png", result)

(355, 474, 3, 2)
dtz= None


C:\Users\HP\AppData\Local\Temp\ipykernel_16632\3141174356.py:73: ComplexWarning: Casting complex values to real discards the imaginary part
  x_next[:,:,0] = x_solver(z[:,:,0,:], u[:,:,0,:], u0_R, rho, deno_R)
C:\Users\HP\AppData\Local\Temp\ipykernel_16632\3141174356.py:75: ComplexWarning: Casting complex values to real discards the imaginary part
  x_next[:,:,1] = x_solver(z[:,:,1,:], u[:,:,1,:], u0_G, rho, deno_G)
C:\Users\HP\AppData\Local\Temp\ipykernel_16632\3141174356.py:77: ComplexWarning: Casting complex values to real discards the imaginary part
  x_next[:,:,2] = x_solver(z[:,:,2,:], u[:,:,2,:], u0_B, rho, deno_B)


TypeError: unsupported operand type(s) for *: 'int' and 'NoneType'